In [ ]:
'''
准备数据
定义模型
模型训练
模型评估
'''

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from torchsummary import summary

import time

In [80]:
def create_dataset():
    raw_data=pd.read_csv('raw_data/手机价格分类预测.csv')
    
    x,y=raw_data.iloc[:,:-1],raw_data.iloc[:,-1]
    
    x=x.astype('float32')
    y=y.astype('int')
    
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100)
    
    dataset_train=TensorDataset(torch.Tensor(x_train.values),torch.IntTensor(y_train.values))
    dataset_test=TensorDataset(torch.Tensor(x_test.values),torch.IntTensor(y_test.values))
    
    return dataset_train,dataset_test,x_train.shape[1],len(pd.unique(y))
dataset_train,dataset_test,a,b=create_dataset()    

In [85]:
class PhoneClassModel(nn.Module):
    def __init__(self):
        super(PhoneClassModel, self).__init__()
        self.layer1=nn.Linear(20,128)
        self.layer2=nn.Linear(128,256)
        self.layer3=nn.Linear(256,512)
        self.out=nn.Linear(512,4)
        
    def forward(self,x):
        y=self.layer1(x)
        y=torch.relu(y)
        y=self.layer2(y)
        y=torch.relu(y)
        y=self.layer3(y)
        y=torch.relu(y)
        y=self.out(y)
        # y=torch.log_softmax(y,dim=1)
        return y
model=PhoneClassModel()
model.cuda()
summary(model,input_size=(1,20))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 128]           2,688
            Linear-2               [-1, 1, 256]          33,024
            Linear-3               [-1, 1, 512]         131,584
            Linear-4                 [-1, 1, 4]           2,052
Total params: 169,348
Trainable params: 169,348
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.65
Estimated Total Size (MB): 0.65
----------------------------------------------------------------


In [117]:
def train():
    torch.cuda.empty_cache()
    torch.manual_seed(42)
    model=PhoneClassModel()
    model.train()
    model.cuda()
    criterion=nn.CrossEntropyLoss()
    optimizer=optim.AdamW(model.parameters(),lr=0.009,betas=(0.9,0.9))
    epochs=700
    loss_list=[]
    for epoch in range(epochs):
        dataloader=DataLoader(dataset_train,batch_size=128,shuffle=True)
        
        start=time.time()
        
        total_loss=0
        total_num=1
        
        for x,y in dataloader:
            x,y=x.cuda(),y.cuda()
            output=model(x)
            y=y.long()
            # print(output,y)
            # print(output.dtype,y.dtype)
            loss=criterion(output,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss=total_loss+loss.item()
            total_num=total_num+1
        loss_list.append(total_loss/total_num)
        print('epoch:%4s loss:%.2f ,time:%.2fs'%(epoch+1,total_loss/total_num,time.time()-start))
    torch.save(model.state_dict(),'model/phone.pth')
train()

epoch:   1 loss:94.52 ,time:0.04s
epoch:   2 loss:1.10 ,time:0.05s
epoch:   3 loss:1.00 ,time:0.03s
epoch:   4 loss:0.95 ,time:0.05s
epoch:   5 loss:0.96 ,time:0.03s
epoch:   6 loss:0.90 ,time:0.02s
epoch:   7 loss:0.87 ,time:0.05s
epoch:   8 loss:0.81 ,time:0.03s
epoch:   9 loss:0.74 ,time:0.04s
epoch:  10 loss:0.69 ,time:0.03s
epoch:  11 loss:0.68 ,time:0.03s
epoch:  12 loss:0.62 ,time:0.03s
epoch:  13 loss:0.61 ,time:0.03s
epoch:  14 loss:0.59 ,time:0.03s
epoch:  15 loss:0.61 ,time:0.03s
epoch:  16 loss:0.56 ,time:0.02s
epoch:  17 loss:0.54 ,time:0.03s
epoch:  18 loss:0.49 ,time:0.03s
epoch:  19 loss:0.47 ,time:0.03s
epoch:  20 loss:0.51 ,time:0.03s
epoch:  21 loss:0.48 ,time:0.03s
epoch:  22 loss:0.47 ,time:0.03s
epoch:  23 loss:0.40 ,time:0.03s
epoch:  24 loss:0.49 ,time:0.03s
epoch:  25 loss:0.42 ,time:0.03s
epoch:  26 loss:0.44 ,time:0.03s
epoch:  27 loss:0.44 ,time:0.03s
epoch:  28 loss:0.39 ,time:0.03s
epoch:  29 loss:0.40 ,time:0.02s
epoch:  30 loss:0.38 ,time:0.03s
epoch:  3

In [119]:
def test():
    model=PhoneClassModel()
    model.load_state_dict(torch.load('model/phone.pth',weights_only=True))
    model.eval()
    dataloader=DataLoader(dataset_test,batch_size=128,shuffle=True)
    correct=0
    with torch.no_grad():
        for x,y in dataloader:
            output=model(x)
            y_pred=torch.argmax(output,dim=-1)
            correct=correct+(y_pred==y).sum().item()
    print(correct/len(dataset_test))     
test()

0.965
